### Problem 3 
The third data set provides a set of TLEs for Planet’s Flock 2B 5 satellite at a series of
epochs. For this question, we’ll use regression methods to predict future BStar (drag)
terms.
1
• Regress BStar on epoch. You choose your regression method(s). Use your results
to predict future BStar terms for epoch 3/20/2019 19:23 to 5/19/2019 20:04,
based on data from epochs 5/11/2018 19:56 to 3/20/2019 3:38. Assess how well
your model fits the data. Plot your results.
• Regress BStar on one or more of the numeric columns in the TLE file. Assess
how well your regression method models the data. Use your results to predict
future BStar terms for the same epochs as above. Assess how well your regression
method models the data. Plot your results.
Hint: Which orbital element(s) would you expect to be most closely associated
with atmospheric drag?


## Imports


In [22]:
import tensorflow as tf
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.regularizers import l1



## Data Parsing and Analysis

In [16]:
df = pd.read_csv("../Data/Week2_Problem3.csv")

In [17]:
df.head()

,COMMENT,ORIGINATOR,NORAD_CAT_ID,OBJECT_NAME,OBJECT_TYPE,CLASSIFICATION_TYPE,INTLDES,EPOCH,EPOCH_MICROSECONDS,MEAN_MOTION,...,FILE,TLE_LINE0,TLE_LINE1,TLE_LINE2,OBJECT_ID,OBJECT_NUMBER,SEMIMAJOR_AXIS,PERIOD,APOGEE,PERIGEE
0,GENERATED VIA SPACETRACK.ORG API,18 SPCS,41616,FLOCK 2P 5,PAYLOAD,U,16040T,5/11/2018 19:56,294304,15.242841,...,2283918,0 FLOCK 2P 5,1 41616U 16040T 18131.83058211 .00001576 0...,2 41616 97.4199 197.0357 0010712 19.7074 340...,2016-040T,41616,6871.072,94.47,500.298,485.577
1,GENERATED VIA SPACETRACK.ORG API,18 SPCS,41616,FLOCK 2P 5,PAYLOAD,U,16040T,5/11/2018 21:30,151904,15.242843,...,2283955,0 FLOCK 2P 5,1 41616U 16040T 18131.89622861 +.00001570 +0...,2 41616 097.4199 197.1007 0010702 019.4876 340...,2016-040T,41616,6871.072,94.47,500.290,485.583
2,GENERATED VIA SPACETRACK.ORG API,18 SPCS,41616,FLOCK 2P 5,PAYLOAD,U,16040T,5/12/2018 14:50,469728,15.242879,...,2284257,0 FLOCK 2P 5,1 41616U 16040T 18132.61833877 +.00001672 +0...,2 41616 097.4199 197.8157 0010632 017.0701 343...,2016-040T,41616,6871.061,94.47,500.231,485.621
3,GENERATED VIA SPACETRACK.ORG API,18 SPCS,41616,FLOCK 2P 5,PAYLOAD,U,16040T,5/12/2018 22:43,705888,15.242887,...,2284359,0 FLOCK 2P 5,1 41616U 16040T 18132.94657067 +.00001604 +0...,2 41616 097.4198 198.1406 0010594 015.9467 344...,2016-040T,41616,6871.059,94.47,500.203,485.644
4,GENERATED VIA SPACETRACK.ORG API,18 SPCS,41616,FLOCK 2P 5,PAYLOAD,U,16040T,5/13/2018 3:26,226848,15.242893,...,2284635,0 FLOCK 2P 5,1 41616U 16040T 18133.14350957 +.00001577 +0...,2 41616 097.4198 198.3356 0010575 015.2670 344...,2016-040T,41616,6871.057,94.47,500.188,485.656


We can see the raw data above. Next, we'll work on transforming the epoch into an integer value, and removing the unnecessary 3 initial columns.(Satellite 41616)

In [18]:
df = df.drop(columns=["COMMENT","ORIGINATOR","OBJECT_TYPE","OBJECT_ID","NORAD_CAT_ID","OBJECT_NAME","ELEMENT_SET_NO","CLASSIFICATION_TYPE","INTLDES"],axis=1)

In [19]:
df.head()

,EPOCH,EPOCH_MICROSECONDS,MEAN_MOTION,ECCENTRICITY,INCLINATION,RA_OF_ASC_NODE,ARG_OF_PERICENTER,MEAN_ANOMALY,EPHEMERIS_TYPE,REV_AT_EPOCH,...,MEAN_MOTION_DDOT,FILE,TLE_LINE0,TLE_LINE1,TLE_LINE2,OBJECT_NUMBER,SEMIMAJOR_AXIS,PERIOD,APOGEE,PERIGEE
0,5/11/2018 19:56,294304,15.242841,0.001071,97.4199,197.0357,19.7074,340.4576,0,10468,...,0,2283918,0 FLOCK 2P 5,1 41616U 16040T 18131.83058211 .00001576 0...,2 41616 97.4199 197.0357 0010712 19.7074 340...,41616,6871.072,94.47,500.298,485.577
1,5/11/2018 21:30,151904,15.242843,0.001070,97.4199,197.1007,19.4876,340.6768,0,10469,...,0,2283955,0 FLOCK 2P 5,1 41616U 16040T 18131.89622861 +.00001570 +0...,2 41616 097.4199 197.1007 0010702 019.4876 340...,41616,6871.072,94.47,500.290,485.583
2,5/12/2018 14:50,469728,15.242879,0.001063,97.4199,197.8157,17.0701,343.0892,0,10480,...,0,2284257,0 FLOCK 2P 5,1 41616U 16040T 18132.61833877 +.00001672 +0...,2 41616 097.4199 197.8157 0010632 017.0701 343...,41616,6871.061,94.47,500.231,485.621
3,5/12/2018 22:43,705888,15.242887,0.001059,97.4198,198.1406,15.9467,344.2102,0,10485,...,0,2284359,0 FLOCK 2P 5,1 41616U 16040T 18132.94657067 +.00001604 +0...,2 41616 097.4198 198.1406 0010594 015.9467 344...,41616,6871.059,94.47,500.203,485.644
4,5/13/2018 3:26,226848,15.242893,0.001057,97.4198,198.3356,15.2670,344.8884,0,10488,...,0,2284635,0 FLOCK 2P 5,1 41616U 16040T 18133.14350957 +.00001577 +0...,2 41616 097.4198 198.3356 0010575 015.2670 344...,41616,6871.057,94.47,500.188,485.656


In [ ]:
model = Sequential()
model.add(Dense(2,
                activation='softmax',
               kernel_regularizer=l1(l1=.1),
               input_dim=len(feature_vector)))